In [57]:
import astropy
import os
import xlsxwriter
import cv2
import tracemalloc
import io
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas  # noqa: E402, E501
from astropy import io
from astropy.io import fits
from astropy import table
from astropy import wcs
from astropy.wcs import WCS
from skimage.transform import resize
from astronomaly.base.base_dataset import Dataset
from astronomaly.base import logging_tools
from astronomaly.data_management.image_reader import ImageDataset
from astronomaly.data_management import image_reader
from astronomaly.preprocessing import image_preprocessing
from astronomaly.feature_extraction import power_spectrum, autoencoder
from astronomaly.feature_extraction import shape_features
from astronomaly.dimensionality_reduction import pca
from astronomaly.postprocessing import scaling
from astronomaly.anomaly_detection import isolation_forest, human_loop_learning
from astronomaly.visualisation import tsne
from astronomaly.utils import utils
from astronomaly.data_management.image_reader import AstroImage

#mpl.use('Agg')

_________

## Convert tractor catalogue

In [242]:
def convert_tractor_catalogue(catalogue_file, image_file, image_name=''):
    """
    Converts a tractor fits file to a pandas dataframe to be given
    directly to an ImageDataset object.

    Parameters
    ----------
    catalogue_files : string
        tractor catalogue in fits table format 
    image_file:
        The image corresponding to this catalogue (to extract pixel information
        and naming information)
    """

    catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

    dataframe = {}
    for name in catalogue.colnames:
        data = catalogue[name].tolist()
        dataframe[name] = data
    
    old_catalogue = pd.DataFrame(dataframe)
    hdul = astropy.io.fits.open(image_file)

    if len(image_name) == 0:
        original_image_ = image_file.split(os.path.sep)[-1]
        original_image = original_image_.replace(original_image_[:2],'')
    else:
        original_image = image_name
    
    new_catalogue = pd.DataFrame()
    new_catalogue['objid'] = old_catalogue['objid']
    new_catalogue['original_image'] = [original_image] * len(new_catalogue)
    new_catalogue['peak_flux'] = old_catalogue['flux_g']
    #new_catalogue['flux_g'] = old_catalogue['flux_g']
    new_catalogue['flux_r'] = old_catalogue['flux_r']
    new_catalogue['flux_z'] = old_catalogue['flux_z']
    new_catalogue['x'] = old_catalogue['bx'].astype('int')
    new_catalogue['y'] = old_catalogue['by'].astype('int')
    new_catalogue['ra'] = old_catalogue['ra']
    new_catalogue['dec'] = old_catalogue['dec']
    
    new_catalogue['type'] = old_catalogue['type']
    catalogue_no_PSFs = new_catalogue[new_catalogue.type.astype("S") != b'PSF ']
    
    return catalogue_no_PSFs

__________

## Create a catalogue for each tractor file

In [243]:
catalogue_file_055 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m055.fits'

image_file_g_055 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m055-image.fits.fz'
image_file_r_055 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m055-image.fits.fz'
image_file_z_055 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m055-image.fits.fz'

image_name = 'legacysurvey-0267m055-image.fits.fz'

pd1 = convert_tractor_catalogue(catalogue_file_055, image_file_z_055, image_name='')

In [244]:
catalogue_file_057 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m057.fits'

image_file_g_057 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m057-image.fits.fz'
image_file_r_057 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m057-image.fits.fz'
image_file_z_057 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m057-image.fits.fz'

image_name = 'legacysurvey-0267m057-image.fits.fz'

pd2 = convert_tractor_catalogue(catalogue_file_057, image_file_z_057, image_name='')

In [245]:
catalogue_file_060 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m060.fits'

image_file_g_060 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m060-image.fits.fz'
image_file_r_060 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m060-image.fits.fz'
image_file_z_060 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m060-image.fits.fz'

image_name = 'legacysurvey-0267m060-image.fits.fz'

pd3 = convert_tractor_catalogue(catalogue_file_060, image_file_z_060, image_name='')

In [246]:
catalogue_file_062 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m062.fits'

image_file_g_062 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m062-image.fits.fz'
image_file_r_062 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m062-image.fits.fz'
image_file_z_062 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m062-image.fits.fz'

image_name = 'legacysurvey-0267m062-image.fits.fz'

pd4 = convert_tractor_catalogue(catalogue_file_062, image_file_z_062, image_name='')

In [247]:
catalogue_file_065 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m065.fits'

image_file_g_065 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m065-image.fits.fz'
image_file_r_065 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m065-image.fits.fz'
image_file_z_065 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m065-image.fits.fz'

image_name = 'legacysurvey-0267m065-image.fits.fz'

pd5 = convert_tractor_catalogue(catalogue_file_065, image_file_z_065, image_name='')

In [248]:
catalogue_file_067 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m067.fits'

image_file_g_067 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m067-image.fits.fz'
image_file_r_067 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m067-image.fits.fz'
image_file_z_067 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m067-image.fits.fz'

image_name = 'legacysurvey-0267m067-image.fits.fz'

pd6 = convert_tractor_catalogue(catalogue_file_067, image_file_z_067, image_name='')

In [249]:
catalogue_file_092 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m092.fits'

image_file_g_092 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m092-image.fits.fz'
image_file_r_092 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m092-image.fits.fz'
image_file_z_092 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m092-image.fits.fz'

image_name = 'legacysurvey-0267m092-image.fits.fz'

pd7 = convert_tractor_catalogue(catalogue_file_092, image_file_z_092, image_name='')

In [250]:
catalogue_file_095 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m095.fits'

image_file_g_095 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m095-image.fits.fz'
image_file_r_095 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m095-image.fits.fz'
image_file_z_095 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m095-image.fits.fz'

image_name = 'legacysurvey-0267m095-image.fits.fz'

pd8 = convert_tractor_catalogue(catalogue_file_095, image_file_z_095, image_name='')

In [251]:
catalogue_file_097 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/tractor-0267m097.fits'

image_file_g_097 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/g-legacysurvey-0267m097-image.fits.fz'
image_file_r_097 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/r-legacysurvey-0267m097-image.fits.fz'
image_file_z_097 = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m097-image.fits.fz'

image_name = 'legacysurvey-0267m097-image.fits.fz'

pd9 = convert_tractor_catalogue(catalogue_file_097, image_file_z_097, image_name='')

______

### Create a single catalogue from the ones above

In [252]:
pdList = [pd1, pd2, pd3,pd4,pd5,pd6,pd7,pd8,pd9]  # List of your dataframes
large_catalogue = pd.concat(pdList)
large_catalogue

,objid,original_image,peak_flux,flux_r,flux_z,x,y,ra,dec,type
0,0,legacysurvey-0267m055-image.fits.fz,4.599823,11.780396,22.910582,967,34,26.797254,-5.628483,EXP
1,1,legacysurvey-0267m055-image.fits.fz,0.891017,2.780183,8.695500,962,73,26.797641,-5.625607,REX
2,2,legacysurvey-0267m055-image.fits.fz,0.676702,1.608683,4.078941,932,117,26.799828,-5.622403,REX
3,3,legacysurvey-0267m055-image.fits.fz,0.042543,0.319040,2.364341,978,57,26.796466,-5.626809,REX
4,4,legacysurvey-0267m055-image.fits.fz,4.975256,9.013798,13.551513,1058,0,26.790606,-5.630930,EXP
...,...,...,...,...,...,...,...,...,...,...
7759,7759,legacysurvey-0267m097-image.fits.fz,0.927698,2.323367,3.712649,1987,3522,26.732593,-9.624581,REX
7761,7761,legacysurvey-0267m097-image.fits.fz,0.021620,0.163782,0.357528,601,3517,26.834893,-9.624991,REX
7763,7763,legacysurvey-0267m097-image.fits.fz,0.097634,0.143542,0.318315,3350,3521,26.632009,-9.624648,REX
7767,7767,legacysurvey-0267m097-image.fits.fz,0.234619,0.391112,0.414931,902,3533,26.812658,-9.623805,EXP


In [253]:
large_catalogue.reset_index(inplace=True)

In [254]:
large_catalogue['objid'] = large_catalogue.index

In [255]:
large_catalogue.to_csv('/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/large_catalogue.csv')

In [256]:
large_catalogue

,index,objid,original_image,peak_flux,flux_r,flux_z,x,y,ra,dec,type
0,0,0,legacysurvey-0267m055-image.fits.fz,4.599823,11.780396,22.910582,967,34,26.797254,-5.628483,EXP
1,1,1,legacysurvey-0267m055-image.fits.fz,0.891017,2.780183,8.695500,962,73,26.797641,-5.625607,REX
2,2,2,legacysurvey-0267m055-image.fits.fz,0.676702,1.608683,4.078941,932,117,26.799828,-5.622403,REX
3,3,3,legacysurvey-0267m055-image.fits.fz,0.042543,0.319040,2.364341,978,57,26.796466,-5.626809,REX
4,4,4,legacysurvey-0267m055-image.fits.fz,4.975256,9.013798,13.551513,1058,0,26.790606,-5.630930,EXP
...,...,...,...,...,...,...,...,...,...,...,...
41409,7759,41409,legacysurvey-0267m097-image.fits.fz,0.927698,2.323367,3.712649,1987,3522,26.732593,-9.624581,REX
41410,7761,41410,legacysurvey-0267m097-image.fits.fz,0.021620,0.163782,0.357528,601,3517,26.834893,-9.624991,REX
41411,7763,41411,legacysurvey-0267m097-image.fits.fz,0.097634,0.143542,0.318315,3350,3521,26.632009,-9.624648,REX
41412,7767,41412,legacysurvey-0267m097-image.fits.fz,0.234619,0.391112,0.414931,902,3533,26.812658,-9.623805,EXP


_____________

## Run it through Astronomaly

### Basic setting for DECaLS data

In [257]:
which_data = 'decals'

coadd_id = '026'

feature_method = 'ellipse'

dim_reduction = ''

band_prefixes = ['z-', 'r-', 'g-']

band_rgb = {'r': 'z-', 'g': 'r-', 'b': 'g-'}

image_transform_function = [image_preprocessing.image_transform_sigma_clipping,
                            # image_preprocessing.image_transform_inverse_sinh,
                            image_preprocessing.image_transform_scale
                            ]

display_transform_function = [#image_preprocessing.image_transform_inverse_sinh,
                              image_preprocessing.image_transform_scale]

plot_cmap = 'hot'
window_size = 32
list_of_files = []

### Reading in catalogue and images

In [261]:
#data_dir = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/'
data_dir = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/'

image_dir = os.path.join(data_dir, 'Images')

#output_dir = os.path.join(
#    '/home/verlon/Desktop/Astronomaly/Data/Output/Coadd026','')

output_dir = os.path.join(
    '/home/verlon/Desktop/Astronomaly/Data/Output/0267 Brick', '')

#catalogue = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/test_catalogue_0260m062_500.csv'
#cat = pd.read_csv(catalogue)
catalogue = large_catalogue

### Running Astronomaly

In [ ]:
image_dataset = image_reader.ImageDataset(directory=image_dir,
                                          list_of_files=list_of_files,
                                          window_size=window_size, 
                                          output_dir=output_dir, 
                                          plot_square=False,
                                          transform_function=image_transform_function,
                                          display_transform_function=display_transform_function,
                                          plot_cmap=plot_cmap,
                                          catalogue=catalogue,
                                          band_prefixes=band_prefixes,
                                          bands_rgb=band_rgb)

Reading image data from /home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m055-image.fits.fz...


2020-10-04 21:04:21,015 - WARNING - FITSFixedWarning: 'datfix' made the change 'Set DATE-REF to '1858-11-17' from MJD-REF'.


Done!
Reading image data from /home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m057-image.fits.fz...
Done!
Reading image data from /home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m060-image.fits.fz...
Done!
Reading image data from /home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Images/z-legacysurvey-0267m062-image.fits.fz...


In [ ]:
cat2 = pd.read_csv(catalogue)
cat2

In [237]:
inds = ['1']

imgs = []

for i in inds:
    imgs.append(image_dataset.get_sample(i))

In [238]:
np.shape(imgs[0])

(32, 32)

In [239]:
def apply_transform(cutout, transform_function):
    """
    Applies the transform function(s) given at initialisation to the image.

    Parameters
    ----------
    cutout : np.ndarray
        Cutout of image

    Returns
    -------
    np.ndarray
        Transformed cutout
    """
    if transform_function is not None:
        try:
            len(transform_function)
            new_cutout = cutout
            for f in transform_function:
                new_cutout = f(new_cutout)
            cutout = new_cutout
        except TypeError:  # Simple way to test if there's only one function
            cutout = transform_function(cutout)
    return cutout

In [240]:
image_dataset.catalogue.columns

Index(['Unnamed: 0', 'objid', 'original_image', 'flux_g', 'flux_r', 'flux_z',
       'x', 'y', 'ra', 'dec', 'type', 'peak_flux'],
      dtype='object')

In [115]:
cat.index = cat.objid

In [232]:
image_dataset.metadata['flux_g'] = image_dataset.catalogue['flux_g']

In [233]:
image_dataset.catalogue['flux_g']

0      317.535095
1      168.212524
2       51.362320
3       71.172943
4       43.115738
          ...    
495      0.981929
496      1.468138
497      0.865858
498      0.940797
499      0.053244
Name: flux_g, Length: 500, dtype: float64

In [241]:
image_dataset.metadata

,original_image,x,y,ra,dec,peak_flux
594,legacysurvey-0260m062-image.fits.fz,337,369,26.126581,-6.354089,NaN
6095,legacysurvey-0260m062-image.fits.fz,994,2486,26.078486,-6.199997,NaN
2709,legacysurvey-0260m062-image.fits.fz,3216,1209,25.915781,-6.292911,NaN
5667,legacysurvey-0260m062-image.fits.fz,1088,2359,26.071583,-6.209265,NaN
2471,legacysurvey-0260m062-image.fits.fz,2859,1137,25.941948,-6.298205,NaN
...,...,...,...,...,...,...
6345,legacysurvey-0260m062-image.fits.fz,292,2585,26.129869,-6.192759,NaN
4612,legacysurvey-0260m062-image.fits.fz,3450,1864,25.898683,-6.245235,NaN
2445,legacysurvey-0260m062-image.fits.fz,1230,1161,26.061230,-6.296397,NaN
3190,legacysurvey-0260m062-image.fits.fz,2916,1365,25.937799,-6.281606,NaN


In [196]:
x0 = image_dataset.metadata.loc['594', 'x']
y0 = image_dataset.metadata.loc['594', 'y']
original_image = image_dataset.metadata.loc['594', 'original_image']
this_image = image_dataset.images[original_image]

In [202]:
x0

337

In [203]:
y0

369

In [204]:
original_image

'legacysurvey-0260m062-image.fits.fz'

In [205]:
this_image

In [206]:
x_wid = image_dataset.window_size_x // 2
y_wid = image_dataset.window_size_y // 2

In [207]:
y_start = y0 - y_wid
y_end = y0 + y_wid
x_start = x0 - x_wid
x_end = x0 + x_wid

invalid_y = y_start < 0 or y_end > this_image.metadata['NAXIS1']
invalid_x = x_start < 0 or x_end > this_image.metadata['NAXIS2']
if invalid_y or invalid_x:
    cutout = np.ones((image_dataset.window_size_y, image_dataset.window_size_x)) * np.nan
else:
    cutout = this_image.get_image_data(y_start, y_end, x_start, x_end)

In [208]:
this_image

In [209]:
y_start

353

In [222]:
this_image.get_image_data(y_start, y_end, x_start, x_end)

array([[[0.06900791, 0.04824349, 0.01793454],
        [0.07997435, 0.04904485, 0.02416287],
        [0.10864868, 0.05419752, 0.0244699 ],
        ...,
        [0.24923168, 0.1379856 , 0.05949469],
        [0.22435555, 0.12911186, 0.05617413],
        [0.22188106, 0.12038541, 0.05671615]],

       [[0.08096706, 0.0413486 , 0.02067803],
        [0.08341669, 0.0473825 , 0.02267906],
        [0.10575445, 0.05389149, 0.02271221],
        ...,
        [0.27469906, 0.15650073, 0.07354525],
        [0.26256877, 0.14644983, 0.06508806],
        [0.23819044, 0.13525721, 0.05740151]],

       [[0.09424462, 0.04744039, 0.02374754],
        [0.09373228, 0.05331422, 0.02656277],
        [0.08829503, 0.05443418, 0.02876392],
        ...,
        [0.28525   , 0.16769014, 0.07698938],
        [0.2571566 , 0.15585926, 0.07177512],
        [0.24595205, 0.14130439, 0.06300192]],

       ...,

       [[0.20277287, 0.13274561, 0.06229659],
        [0.21953629, 0.13621432, 0.06209242],
        [0.24997954, 0

In [211]:
image_dataset.metadata

,original_image,x,y,ra,dec,peak_flux
594,legacysurvey-0260m062-image.fits.fz,337,369,26.126581,-6.354089,NaN
6095,legacysurvey-0260m062-image.fits.fz,994,2486,26.078486,-6.199997,NaN
2709,legacysurvey-0260m062-image.fits.fz,3216,1209,25.915781,-6.292911,NaN
5667,legacysurvey-0260m062-image.fits.fz,1088,2359,26.071583,-6.209265,NaN
2471,legacysurvey-0260m062-image.fits.fz,2859,1137,25.941948,-6.298205,NaN
...,...,...,...,...,...,...
6345,legacysurvey-0260m062-image.fits.fz,292,2585,26.129869,-6.192759,NaN
4612,legacysurvey-0260m062-image.fits.fz,3450,1864,25.898683,-6.245235,NaN
2445,legacysurvey-0260m062-image.fits.fz,1230,1161,26.061230,-6.296397,NaN
3190,legacysurvey-0260m062-image.fits.fz,2916,1365,25.937799,-6.281606,NaN


In [213]:
if image_dataset.metadata.loc['594', 'peak_flux'] == -1:
    if np.any(np.isnan(cutout)):
        flx = -1
    else:
        flx = np.max(cutout)
    image_dataset.metadata.loc['594', 'peak_flux'] = flx

cutout = apply_transform(cutout, image_dataset.transform_function)

In [214]:
cutout

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

In [133]:
x_end

353

In [134]:
cutout

array([[[0.06900791, 0.04824349, 0.01793454],
        [0.07997435, 0.04904485, 0.02416287],
        [0.10864868, 0.05419752, 0.0244699 ],
        ...,
        [0.24923168, 0.1379856 , 0.05949469],
        [0.22435555, 0.12911186, 0.05617413],
        [0.22188106, 0.12038541, 0.05671615]],

       [[0.08096706, 0.0413486 , 0.02067803],
        [0.08341669, 0.0473825 , 0.02267906],
        [0.10575445, 0.05389149, 0.02271221],
        ...,
        [0.27469906, 0.15650073, 0.07354525],
        [0.26256877, 0.14644983, 0.06508806],
        [0.23819044, 0.13525721, 0.05740151]],

       [[0.09424462, 0.04744039, 0.02374754],
        [0.09373228, 0.05331422, 0.02656277],
        [0.08829503, 0.05443418, 0.02876392],
        ...,
        [0.28525   , 0.16769014, 0.07698938],
        [0.2571566 , 0.15585926, 0.07177512],
        [0.24595205, 0.14130439, 0.06300192]],

       ...,

       [[0.20277287, 0.13274561, 0.06229659],
        [0.21953629, 0.13621432, 0.06209242],
        [0.24997954, 0

In [135]:
plt.figure(figsize = (8,8))
imgplot = plt.imshow(image_dataset.get_sample('1'))
plt.show()

KeyError: '1'

In [136]:
from PIL import Image

In [168]:
img = Image.fromarray(imgs[0])
img.show()